# Домашнее задание 6: классификация текстов

Мы будем работать с датасетом объявлений Avito. Нам предстоит по заголовку и тексту объявления предсказывать категорию объявления.

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('avito-text-classification/train.csv')
test = pd.read_csv('avito-text-classification/test.csv')

In [4]:
train.head()

,title,description,Category_name,Category
0,"Эбу Renault / Nissan 1,5 DCI Delphi 2001-2008 год","Комплект ЭБУ (мозги, компьютер мотора, двигате...",Запчасти и аксессуары,10
1,Утюг утп 1000 ватт СССР 1987 год,Продам/\n Фото № 1-2 /\n /\nУтюг УТП 1000 ватт...,Бытовая техника,21
2,Возвму машину с выкупом,Возьму машину в аренду с последующим выкупом н...,Предложение услуг,114
3,Полусапожки,"полусапожки в отличном состоянии, один раз оде...","Одежда, обувь, аксессуары",27
4,Босоножки кожаные,"Кожаные(натур) босоножки Karlo Pasolini, 40 рр...","Одежда, обувь, аксессуары",27


In [3]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [5]:
train['text'] = train['title'] + ' ' + train['description']
train.drop(['title', 'description'], axis=1, inplace=True)

In [6]:
test['text'] = test['title'] + ' ' + test['description']
test.drop(['title', 'description'], axis=1, inplace=True)

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from string import punctuation, digits

def tokenizer(s: str) -> list:
    """
    My custom tokenizer using sets
    """
    
    stemmer = SnowballStemmer('russian')
    compare = set(punctuation) | set(digits) | set(stopwords.words('russian'))
    result = [stemmer.stem(x) for x in word_tokenize(s.lower()) if len(x) > 2 and len({x} & compare) == 0]
  
    return result
    

In [9]:
# 1
random_data = train.sample(100000)

In [10]:
random_data

,Category_name,Category,text
4142640,Посуда и товары для кухни,87,"Чайный сервиз з-д «Пролетарий» Чайный сервиз, ..."
323844,Детская одежда и обувь,29,Костюм Крокид зима 104-110 Продам костюм Кроки...
1064444,Детская одежда и обувь,29,"Ботинки Ботинки в отличном состоянии, натур.кожа"
4146547,Детская одежда и обувь,29,Пижамы в комплекте-2шт Описание прилагается к ...
736215,Предложение услуг,114,"Грумер (стрижка собак и кошек) Здравствуйте, /..."
...,...,...,...
3163615,Запчасти и аксессуары,10,Mazda 6 mps подкрылок правый оригинал Без дефе...
3252818,Спорт и отдых,39,Связки и суставы NOW Hyaluronic Acid with MSM ...
902410,"Одежда, обувь, аксессуары",27,"Платье Платье с накидкой. Вечернее, облегающее."
1003642,Автомобили,9,"Mazda CX-7, 2011 Отличное внешнее и техническо..."


In [11]:
# 2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(random_data['text'], random_data['Category'], test_size=0.2, random_state=2021)

In [12]:
%%time

# 3, 4

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report,accuracy_score

sgd = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenizer)),
    ('clf', SGDClassifier(loss='modified_huber', random_state=2021, n_jobs=8)),
    ])

sgd.fit(X_train, y_train)

sgd_y_pred = sgd.predict(X_test)


# classification_report(y_test, sgd_y_pred)
accuracy_score(y_test, sgd_y_pred)

CPU times: user 3min 23s, sys: 6.77 s, total: 3min 30s
Wall time: 2min 55s


0.78865

In [15]:
print("test:", accuracy_score(y_test, sgd_y_pred))

print(classification_report(y_test, sgd_y_pred))

test: 0.78865
              precision    recall  f1-score   support

           9       0.93      0.82      0.87       608
          10       0.87      0.95      0.91      3564
          11       0.86      0.55      0.67        22
          14       0.80      0.40      0.53        40
          19       0.76      0.58      0.66       614
          20       0.79      0.72      0.75       773
          21       0.78      0.62      0.69       341
          23       0.86      0.83      0.85        53
          24       0.99      0.97      0.98       695
          25       1.00      0.96      0.98       228
          26       0.97      0.98      0.97       113
          27       0.68      0.90      0.77      3573
          28       0.89      0.76      0.82       317
          29       0.73      0.86      0.79      2680
          30       0.79      0.72      0.75      1004
          31       0.89      0.65      0.75        62
          32       0.84      0.61      0.71       371
          33 